<a href="https://colab.research.google.com/github/TerryTian21/PyTorch-Practice/blob/main/PyTorch_Video_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Going Modular

 ## What is going modular?

 Going modular involves turning notebook code (Jupyter Notebooks) into a series of different Python scripts that offer similar functionality.

 For example, we could turn our notebook code from a series of cells into pythone files

 * `data_setup.py` - Prepare and download data if needed
 * `engine.py` - Training functions
 * `model_builder.py` - Creates desired Pytorch models

Note: Naming of files depends on use case and code requirements.

## Why go Modular

While notebooks are great for exploration purposes, larger scale porjects may find Python scripts more reproducible and easier to run.

*Production Code* is code that runs to offer a service to someone. Libraries like `nb-dev` allow you to write Python libraries in Jupyer Notebooks.

Usually, machine learning projects start in notebooks and the useful pieces are moved into Python scripts once they are confirmed to be workign as intended.


## Things to Note

**Docstrings** - Writing reproducible and understandable code is important and functions should be created with docstrings.

**Imports at the top** - All scripts require import modules be imported at the start of the script

# Moving Cells to Scripts

## Get Data

Here we create a script for obtaining the image data for our model

In [1]:
import os
import requests
import zipfile
from pathlib import Path

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, and and prepare
if image_path.is_dir():
  print(f"Path Exists")
else:
  print(f"Creating Image Folder")
  image_path.mkdir(parents=True, exist_ok=True)

# Download Dataset
with open(data_path / "images.zip", "wb") as f:
  response = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  f.write(response.content)

# Unzip images
with zipfile.ZipFile(data_path / "images.zip", "r") as zip_ref:
  zip_ref.extractall(data_path)

# Remove zip file
os.remove(data_path / "images.zip")

Creating Image Folder


In [2]:
import os

os.makedirs("going_modular", exist_ok=True)

## Create Dataset and DataLoader

We can convert the dataset and dataloader creation code into a function called create_dataloaers()

We can write it to a file by using the line `%%writefile`

In [3]:
%%writefile going_modular/data_setup.py
"""
Contains functions for create Python Dataloaders for image classification
"""

import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaers(train_dir: str, test_dir:str, transfrom: transforms.Compose, batch_size: int, num_workers: int=NUM_WORKERS):
  """ Creates training and testing DataLoaders

  takes in training/testing directory path and turns them into Datasets and DataLoaders

  Args:
    train_dir (str): Path to training directory
    test_dir (str): Path to testing directory
    transfrom (transforms.Compose): transform to perform on training and test data
    batch_size (int): number of samples per batch in datalaoder
    num_workers (int, optional): Number of workers for dataloader. Defaults to NUM_WORKERS

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names) where class_names is a list of possible labels
    Example usage:
      train_dataloader, test_dataloader, class_names = create_dataloaers(train_dir=path/to/train, test_dir=path/to/test, transfrom, batch_size=32, num_workers=4)

  """

  #Use Image folder to create datasets

  train_dataset = datasets.ImageFolder(train_dir, transform=transfrom)
  test_dataset = datasets.ImageFolder(test_dir, transform=transfrom)

  #Turn image into dataloaders

  train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
  test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

  #Get class names
  class_names = train_dataset.classes

  return train_dataloader, test_dataloader, class_names


Writing going_modular/data_setup.py


If we want to create dataloader's we can just import the function from `data_setup.py`

In [4]:
# Import function
from going_modular import data_setup


## Making a Model

We have repeatedly used the TinyVGG model in the past and so to avoid repetition we can create a script for it

In [5]:
%%writefile going_modular/model_builder.py
"""
Creates an instance of the ResNet34 architecture
"""

import torch
from torch import nn

class ResNet34(nn.Module):
  """Creates the ResNet34 architecture

  Replicates the ResNet34 Architecture from PyTorch

  Args:
    input_shape: An integer representing the number of channels in the input image
    hidden_units: An integer representing the number of nodes inbetween layers
    output_shape: An integer representing the number of classes
    layers: A list of the number of blocks in each layer

  Returns:
    A PyTorch model

  """

  class BasicBlock(nn.Module):

    """ A basic building block of the ResNet34 architecture

    Consists of two convolutional layers with batch normalization and ReLU activations.
    This is a residual block where the input to the convolutions is added to the output of the convolutions

    Args:
      in_features: The input shape of the data
      out_features: The output shape of the data
      stride: The stride of the convolutions
      downsample: An optional downsampling layer for when the stride is not 1

    """

    def __init__(self, in_features, out_features, stride=1, downsample=None):
      super().__init__()
      self.conv = nn.Conv2d(in_features, out_features, kernel_size=3, stride=stride, padding=1)
      self.bn = nn.BatchNorm2d(out_features)
      self.relu = nn.ReLU(inplace=True)
      self.downsample = downsample

    def forward(self, x):

      identity = x

      out = self.conv(x)
      out = self.bn(x)
      out = self.relu(out)
      out = self.conv(out)
      out = self.bn(out)

      if self.downsample is not None:
        identity = self.downsample(x)

      out += identity
      out = self.relu(out)

      return out

  # Init function of the Resnet Model
  def __init__(self, input_shape, hidden_units, output_shape, layers):
    super().__init__()
    self.hidden_units = hidden_units
    self.layers=layers

    self.conv_block = nn.Sequential(
        nn.Conv2d(input_shape, hidden_units, kernel_size=7, stride=2, padding=3, bias=False),
        nn.BatchNorm2d(num_features=hidden_units),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    )

    self.layer1 = self.make_layers(hidden_units, layers[0], stride=1)
    self.layer2 = self.make_layers(hidden_units*2, layers[1], stride=2)
    self.layer3 = self.make_layers(hidden_units*4, layers[2], stride=2)
    self.layer4 = self.make_layers(hidden_units*8, layers[3], stride=2)
    self.avgpool = nn.AdaptiveAvgPool2d((1,1))
    self.classifier=nn.Linear(512, output_shape)

  # Sets the number of blocks in a layer
  def make_layers(self, out_features, blocks, stride):
    downsample = None

    if stride != 1 or self.hidden_units != out_features:
      downsample = nn.Sequential(
          nn.Conv2d(self.hidden_units, out_features, kernel_size=1, stride=2, bias=False),
          nn.BatchNorm2d(out_features)
      )

    layers = []

    # The first block with custom stride
    layers.append(self.BasicBlock(self.hidden_units, out_features, stride, downsample))
    self.hidden_units = out_features

    # Each block after has default stride value
    for _ in range(1, blocks):
      layers.append(self.BasicBlock(self.hidden_units, out_features))

    return nn.Sequential(*layers)


  def forward(self, x):
    x = self.conv_block(x)
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.avgpool(x)
    x = torch.flatten(x, 1)
    return self.classifier(x)

Writing going_modular/model_builder.py


In [6]:
import torch
from going_modular import model_builder

class_names = ["pizza", "steak", "sushi"]

torch.manual_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Instantiate Model
layers = [3, 4, 6, 3]
model = model_builder.ResNet34(input_shape=3, hidden_units=64, layers=layers, output_shape=len(class_names))

In [7]:
model.to(device)

ResNet34(
  (conv_block): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (2): BasicBlock(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_st

## Creating Train and Test Functions

We previously functionized the training loop into three different functions:

* `train_step()`
* `test_step()`
* `train()`

Since these will be the engine of our model we can put them into a Python script called `engine.py`

In [10]:
%%writefile going_modular/engine.py
"""
Contains functions for training and testing a PyTorch model.
"""


import torch
from tqdm.auto import tqdm
from torch import nn

def train_step(mode, dataloader, loss_fn, optimizer, device):
  """
  Train's a Pytorch model for a single epoch.

  Args:
    model: A PyTorch model to be trained.
    dataloader: A PyTorch dataloader for training data
    loss_fn: The loss function
    optimizer: The optimizer used to minimized the loss function
    device: The target device used for computations

  Returns:
    A tuple of training loss and training accuracy metrics
  """

  #Put model in trian mode

  model.train()

  train_loss, train_acc = 0, 0
  for (X,y) in dataloader:
    X, y = X.to(device), y.to(device)

    y_logits = model(X)
    loss = loss_fn(y_logits, y)
    y_preds = y_logits.softmax(dim=1).argmax(dim=1)


    train_loss += loss
    train_acc += ((y_preds == y).sum().item() / len(y_preds))

    # Optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  train_loss, train_acc = train_loss / len(dataloader), train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(model, dataloader, loss_fn, device):
  """
  Tests a PyTorch model for one epoch

  Args:
    model: A PyTorch model to be trained.
    dataloader: A PyTorch dataloader for testing data
    loss_fn: The loss function
    device: The target device used for computations

  Returns:
    A tuple of test loss and test accuracy metrics
  """
  model.eval()

  test_loss, test_acc = 0, 0
  with torch.infernce_mode():
    for (X,y) in dataloader:
      X, y = X.to(device), y.to(device)

      y_logits = model(X)
      test_loss += loss_fn(y_logits, y)

      y_preds = y_logits.softmax(dim=1).argmax(dim=1)
      test_acc += ((y_preds == y_logits).sum().item() / len(y_preds))

    test_loss, test_acc = test_loss / len(dataloader), test_acc / len(dataloader)

  return test_loss, test_acc


def train(model, train_dataloader, test_dataloader, loss_fn, optimizer, epochs, device):
  """
  Trains and Tests A Pytorch Model

  Passes a target model through train_step() and test_step() for a number of epochs.

  Returns:
    A dictionary of training and testing loss and acc metrics.

  """
  print(f" Starting Training ----------------------")
  results = {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": []}

  for epoch in tqdm(range(epochs)):

    train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer, device)
    test_loss, test_acc = test_step(model, test_dataloader, loss_fn, device)

    results["train_loss"] += train_loss
    results["train_acc"] += train_acc
    results["test_loss"] += test_loss
    results["test_acc"] += test_acc

    print(f"Epoch: {epoch} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc:.2f} | Test Loss: {test_loss:.3f} | Test Acc: {test_acc:.2f}")


  return results



Writing going_modular/engine.py


## Util Functions

Util functions that come in handy such as saving and loading a model

In [16]:
%%writefile going_modular/utils.py
"""
Contains various utility functions
 """

import torch
from pathlib import Path

def save_model(model, target_dir, model_name):
  """
  Saves a PyTorch model to a target directory

  Args:
    model: A target PyTorch model to save
    target_dir: Path the the directory
    modle_name: name for model

  """

  # Create target directory

  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True, exists_ok=True)

  # Create Model Save Path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name shoud end in '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save Model
  print("Saving Model")
  torch.save(obj=model.state_dict(), f=model_save_path)

Overwriting going_modular/utils.py


## Putting it Together

Now that we have all of our Python files we can combine them to create a single script `train.py`

This way, we can train our PyTorch model with a single line of code. The steps for doing so are

1. Import the necessary dependcy files (including the previous modules)
2. Set up hyperparameters
3. Set up train and test directories
4. Create necessary data transforms
5. Combine modules to train the model

In [17]:
%%writefile going_modular/train.py
"""
Trains a Pytorch image classification model and stores in models directory
"""

import os
import torch
import data_setup, engine, model_builder, utils

from torchvision.transforms import v2

# Setup Hyperparameters

EPOCHS = 5
BATCH_SIZE = 64
HIDDEN_UNITS=64
LEARNING_RATE = 0.001
LAYERS= [3,4,6,3]

# Directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Transforms
transform = v2.Compose([v2.PILToTensor(),
                        v2.Resize((224,224), antialias=True)])

# Get the dataloaders

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaers(train_dir, test_dir, transform, BATCH_SIZE)

# Create mdoel
model = model_builder.ResNet34(input_shape=3, hidden_units=HIDDEN_UNITS, output_shape=len(class_names), layers=LAYERS).to(device)

# Create loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=LEARNING_RATE)
#Train
engine.train(model, train_dataloader, test_dataloader, loss_fn, optimizer, EPOCHS, device)

# Save the model
utils.save_model(model, "models", "ResNet34.pth")


Writing going_modular/train.py
